In [1]:
from clutter_filter import GroundClutterFilter
import numpy as np
import glob, os
# import radarkit
import radarkitIQ as radarkit
import momentgen as mg

In [2]:
fname = '../data/20240524/RAXPOL-20240524-000839.083.rkc'
# rkid = radarkit.open(fname)
# out = rkid.read()
rkid = radarkit.rkcfile(fname)
print(rkid.pulses)

Filename: ../data/20240524/RAXPOL-20240524-000839.083.rkc
preface = RadarKit/IQ   buildNo = 7
Pulse data size: 986553792
gateCount = 12458   capacity = 36000   downSampledGateCount = 1246
data offset = 4792
Reading pulses ...
Number of pulses: 49014
[( 74031, 101289, 18507750, 3311, 36000, 12458, 1246, 10, 33, 1716509319,  83389, 86919.0868477 , [113,   0,   0,   0], [ 0, 33,  0,  0], 3, 0, 0, 3., 0.18127692, 338.99963, -0.01098648, -6.449073 , [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [[[ 1.72723608e+03, -5.12794922e+03], [-2.38435699e+02,  8.28516235e+01], [-4.36393776e+01, -3.22552567e+01], ..., [ 1.89736032e+00, -1.70763168e+01], [-3.79474640e+00, -5.05964279e+00], [-3.79477215e+00,  1.77087612e+01]], [[-1.46457725e+04, -8.52739844e+03], [-5.03434601e+02, -5.86286255e+02], 

In [3]:
# ciq = out['ciq']

# pulses_new = []
# print(np.array(pulses[0]['iq'][0]))
# for pulse in pulses:
	# pulses_new.append(pulse['iq'])
# pulses = np.array(pulses_new)

pulses = rkid.pulses['iq']
# (pulse, iq, gate, hv)
pulses = pulses.transpose((1,2,3,0))
# (iq, gate, hv, pulse)
pulses = pulses[0,:,:,:] + 1.j*pulses[1,:,:,:]
# (gate, hv, pulse)
pulses = pulses.transpose((0,2,1)).astype(np.complex128)
# (gate, pulse, hv)

# ciq = pulses

# lamb = rkid.desc.wavelength
# prt = radarkit.read_RKFloat_array(rkid.configs.contents.prt,1)
lamb = rkid.header['desc']['wavelength']
prt = rkid.header['config']['prt']
c = 299,792,458
sample_freq = int(rkid.header['waveform']['fs'])
sample_time = 1/sample_freq
va = lamb/(4*prt)
# C = np.abs(radarkit.read_RKComplex_array(rkid.configs.contents.systemZCal, 1)[0])
# l = np.abs(radarkit.read_RKComplex_array(rkid.configs.contents.systemZCal, 2)[1])

totaln = pulses.shape[1]
count = 100 # Pulses per ray
rayCount = int(np.floor(totaln/count)) # Rays per sweep
gateCount = pulses.shape[0]
n = rayCount * count

az = rkid.pulses['azimuthDegrees']
el = rkid.pulses['elevationDegrees']

# # he's expecting ciq to be in (pulse, h/v, range)
# X_h = np.reshape(ciq[:n, 0, :], (rayCount, count, -1))
# # transpose (ray, pulse, range(?)) to (range, ray, pulse)
# X_h = np.transpose(X_h, (2, 0, 1))

# X_v = np.reshape(ciq[:n, 1, :], (rayCount, count, -1))
# X_v = np.transpose(X_v, (2, 0, 1))

X_h = pulses[:,:n,0]
# (gate, pulse)
X_h = np.reshape(X_h, (gateCount, rayCount, count))
# (gate, ray, pulse)

X_v = pulses[:,:n,1]
X_v = np.reshape(X_v, (gateCount, rayCount, count))

az = np.reshape(az[:n], (rayCount, count))
az = az[:,0]
el = np.reshape(el[:n], (rayCount, count))
el = np.nanmean(el, axis=1)


In [4]:
"""# set some imporatnt radar parameter
wavelength_scalar = lamb
# prt_scalar = prt[0]
num_samples_actual = X_h.shape[2]

#filter the I/Q time series for the horizontal and vertical polarizations
filter_inst = GroundClutterFilter(
	wavelength=lamb,
	scan_rate=1/(count * prt),
/tprt=prt,
	num_samples=count
)

print("\n--- Filtering X_h (Horizontal Polarization) ---")
filtered_i_h, filtered_q_h, poly_order_map_h, clutter_i_trend_map_h, clutter_q_trend_map_h, interpolated_spectrum_map_h = \
	filter_inst.filter_iq_data(X_h.real, X_h.imag, cnr_db_map=None, apply_interpolation=True)

filtered_X_h_optimized = filtered_i_h + 1j * filtered_q_h

del filtered_i_h, filtered_q_h, poly_order_map_h, clutter_i_trend_map_h, clutter_q_trend_map_h, interpolated_spectrum_map_h

print("\n--- Filtering X_v (Vertical Polarization) ---")
filtered_i_v, filtered_q_v, poly_order_map_v, clutter_i_trend_map_v, clutter_q_trend_map_v, interpolated_spectrum_map_v = \
	filter_inst.filter_iq_data(X_v.real, X_v.imag, cnr_db_map=None, apply_interpolation=True)

filtered_X_v_optimized = filtered_i_v + 1j * filtered_q_v

del filtered_i_v, filtered_q_v, poly_order_map_v, clutter_i_trend_map_v, clutter_q_trend_map_v, interpolated_spectrum_map_v

print("\nOptimization complete. Filtered data is in filtered_X_h_optimized and filtered_X_v_optimized.")"""

'# set some imporatnt radar parameter\nwavelength_scalar = lamb\n# prt_scalar = prt[0]\nnum_samples_actual = X_h.shape[2]\n\n#filter the I/Q time series for the horizontal and vertical polarizations\nfilter_inst = GroundClutterFilter(\n\twavelength=lamb,\n\tscan_rate=1/(count * prt),\n/tprt=prt,\n\tnum_samples=count\n)\n\nprint("\n--- Filtering X_h (Horizontal Polarization) ---")\nfiltered_i_h, filtered_q_h, poly_order_map_h, clutter_i_trend_map_h, clutter_q_trend_map_h, interpolated_spectrum_map_h = \tfilter_inst.filter_iq_data(X_h.real, X_h.imag, cnr_db_map=None, apply_interpolation=True)\n\nfiltered_X_h_optimized = filtered_i_h + 1j * filtered_q_h\n\ndel filtered_i_h, filtered_q_h, poly_order_map_h, clutter_i_trend_map_h, clutter_q_trend_map_h, interpolated_spectrum_map_h\n\nprint("\n--- Filtering X_v (Vertical Polarization) ---")\nfiltered_i_v, filtered_q_v, poly_order_map_v, clutter_i_trend_map_v, clutter_q_trend_map_v, interpolated_spectrum_map_v = \tfilter_inst.filter_iq_data(X_

In [5]:
noise = rkid.header['config']['noise']
N_h, N_v = noise 
if rkid.header['buildNo'] >= 4:
	if rkid.header['dataType'] == 'raw':
		dr = rkid.header['config']['pulseGateSize']
		# dt = dr * 2 / (c/1e6)
	elif rkid.header['dataType'] == 'compressed':
		dr = rkid.header['desc']['pulseToRayRatio'] * rkid.header['config']['pulseGateSize']
		# dt = dr * 2 / (c/1e6)
	else:
		print("Inconsistency detected. This should not happen.")
		dr = 30.
		# dt = 1./50
else:
	dr = 30.
	# dt = 1./50

# rr = (dr/2) + (np.arange(0,ng,gstep) * dr)

R = np.arange(0, gateCount)*dr
C = rkid.header['config']['ZCal']
Cd = rkid.header['config']['DCal']
Cp = rkid.header['config']['PCal']

moments = mg.get_moments(X_h, X_v, N_h, N_v, R, va, C, Cd, Cp)
# moments_f = mg.get_moments(filtered_X_h_optimized, filtered_X_v_optimized, N_h, N_v, R, va, C, Cd, Cp)

/Users/jwt/Documents/School/5. JuniorFall/WRT/Final/clutter_filter/momentgen.py:64: RuntimeWarning: invalid value encountered in log10
  ZDROUT[it,ir] = 10*np.log10(S_h/S_v) + 10*np.log10(Cd if not Cd==0 else 1e-20)
/Users/jwt/Documents/School/5. JuniorFall/WRT/Final/clutter_filter/momentgen.py:65: RuntimeWarning: invalid value encountered in sqrt
  RHOHVOUT[it,ir] = np.abs(ccf(X_h, X_v, 0))/np.sqrt(S_h*S_v)
/Users/jwt/Documents/School/5. JuniorFall/WRT/Final/clutter_filter/momentgen.py:61: RuntimeWarning: invalid value encountered in log10
  DBZOUT[it,ir] = 10*np.log10(S_h) + 20*np.log10(r if not r==0 else 1e-20) + 10*np.log10(C if not C==0 else 1e-20)
/Users/jwt/Documents/School/5. JuniorFall/WRT/Final/clutter_filter/momentgen.py:63: RuntimeWarning: invalid value encountered in log
  WIDTHOUT[it,ir] = (np.sqrt(2)*va/np.pi)*np.sqrt(np.abs(np.log(S_h/np.abs(acf(X_h,1)))))


In [ ]:
from raxpolCf import raxpolCf

fname_out = '../data/out/20240524/RAXPOL-20240524-000839.nc'

rcf = raxpolCf()
rcf.setVolume()
rcf.setSweep()
rcf.setTime((rkid.pulses['time_tv_sec'][:n:count]).astype(np.float64))
rcf.setRange(R.astype(np.float32))
rcf.setPosition(np.nanmean(rkid.header['desc']['latitude']), np.nanmean(rkid.header['desc']['longitude']))
rcf.setScanningStrategy('ppi')
rcf.setTargetAngle(np.nanmean(rkid.pulses['elevationDegrees']))
rcf.setAzimuth(az)
rcf.setElevation(el)
rcf.setPulseWidthSeconds((rkid.pulses['pulseWidthSampleCount'][:n:count]*sample_time).astype(np.float32))
rcf.setPrtSeconds(np.tile(prt, (rayCount,1)))
rcf.setWavelengthMeters(np.tile(prt, (rayCount,1)))

print(moments['ZDR'])
rcf.setDBZ(moments['DBZ'])
rcf.setVEL(moments['VEL'])
rcf.setWIDTH(moments['WIDTH'])
rcf.setZDR(moments['ZDR'])
rcf.setRHOHV(moments['RHOHV'])
rcf.setPHIDP(moments['PHIDP'], units='degrees')
rcf.setSNRH(moments['SNRH'])
rcf.setSNRV(moments['SNRV'])

rcf.saveToFile(fname_out)

[[-196.55476913 -199.1216006  -184.27667233 ... -211.23726573
  -214.64926712 -207.91724028]
 [-196.57054891 -198.84888189 -189.73786135 ... -216.09798568
  -202.31144515           nan]
 [-196.55761008 -198.72984298 -190.98474979 ... -221.10364429
  -195.13180197           nan]
 ...
 [-196.58196099 -197.30074044 -195.13254303 ...           nan
  -210.16930961           nan]
 [-196.59150836 -198.15150529 -196.47155526 ...           nan
            nan -208.73959425]
 [-196.57779007 -197.41777374 -199.93200768 ...           nan
            nan -215.67309209]]
